In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from scipy import stats
import itertools
import os

## Read and transform data

In [ ]:
#read data
#K:\2020_2431\FX valuation model 2020\g10
os.chdir('K:/2020_2431/FX valuation model 2020/g10/')

df = pd.read_excel(r'K:\2020_2431\FX valuation model 2020\g10\Python FX Data v4.xlsx')

In [ ]:
#read mapping
map = pd.read_csv(r'K:\2020_2431\FX valuation model 2020\g10\Python FX Data Map v3.1.csv')

In [ ]:
#set date index
df['Dates'] = pd.to_datetime(df.Dates)
df.set_index('Dates', inplace=True)

In [ ]:
#common data transformations
df['brent'] = df['co1 comdty']
df['10Y BTP spread'] = df['gtitl10y govt'] - df['gtdem10y govt']
df['US 2s10s'] = df['ussa10 curncy'] - df['ussa2 curncy']

#create 2Y swap series for each G10 pair
for m in range(len(map)):
    df['2Y spread ' + map['g10_pairs'][m]] = df[map['twoy_swap_1'][m]] - df[map['twoy_swap_2'][m]]

## Define functions

In [ ]:
#regress full time horizon
def regress(Y, X):

    #clean the data
    Y = Y.dropna()
    X = X.dropna()
    
    #make Y matrix the same shape as X
    YX = pd.concat([Y, X], axis=1).reindex(X.index).dropna()
    Y = YX.iloc[:,0]
    X = YX.drop(YX.columns[0], axis = 1)
    
    print(YX.corr())
    
    #fit the model with a constant
    X1 = sm.add_constant(X)
    reg = sm.OLS(Y, X1).fit()
    print(reg.summary())

    #calculate and plot residuals
    predictions = reg.predict()
    residuals = predictions - Y
   
    plt.figure(figsize=(6,3))
    plt.plot(residuals, color = 'blue')
    
    plt.title('Residuals', fontsize = 16)
    
    for year in range(Y.index[0].year, Y.index[-1].year+1):
        plt.axvline(datetime(year,1,1), linestyle = '--', color = 'k', alpha = 0.5)
    plt.show()
    
    #create exportable dataframe for actual data, predictions and residuals
    predictions = pd.Series(predictions, name = 'Predictions', index = Y.index)
    results = pd.DataFrame(predictions, index = Y.index)
    results['Actual'] = Y
    results['Residuals'] = results['Predictions'] - results['Actual']
    
    #plot actual data vs. predictions
    predictions = pd.Series(predictions, index = Y.index)
    plt.figure(figsize=(20,7))
    plt.plot(Y)
    plt.plot(predictions)
    plt.legend(('Actual', 'Model Fit'), fontsize = 20)
    plt.title('Full Year Model Fit', fontsize = 20)
    
    for year in range(predictions.index[0].year, predictions.index[-1].year+1):
        plt.axvline(datetime(year,1,1), linestyle = '--', color = 'k', alpha = 0.5)
    plt.show()

In [ ]:
#rolling regression (for 3m, 6m, 1y, 2y, 4y horizons) WIP
def rolling_reg(Y, X, ccy_id, model_id):
    
    #clean data
    Y = Y.dropna()
    X = X.dropna()

    #make Y matrix the same shape as X
    YX = pd.concat([Y, X], axis=1).reindex(X.index).dropna()

    #create list for timeframes where numbers are days in the window (3m = 63; 6m = 126; 1y = 252; 2y = 504; 4y = 1008)
#     time_frame_id = ['3m', '6m', '1y', '2y', '4y', 'grow']
#     time_frames = [63, 126, 252, 504, 1008, 63]

    time_frame_id = ['2y']
    time_frames = [504]
    
    #create dataframe for results
    results = pd.DataFrame(index = Y.index)
    
    #loop for time frames
    for tf in range(len(time_frames)):
        time_frame = time_frame_id[tf]
        m = time_frames[tf]

        #create series for rolling predictions
        Y_test = YX[YX.index[m]:]
        Y_test = Y_test[Y_test.columns[0]]
        rolling_predictions = np.zeros_like(Y_test)
        rolling_predictions = pd.Series(index = Y_test.index)

        #create series for rolling intercepts, betas, and r-squared
        intercept = np.zeros_like(Y_test)
        beta = pd.DataFrame()
        for x in X.columns:
            beta[ccy_id + '_' + time_frame + '_' + model_id + '_' + 'beta_' + x] = np.zeros_like(Y_test)
        beta = beta.reindex(Y_test.index)
        r2 = np.zeros_like(Y_test)

        #generate rolling predictions
        for t in range(m, len(YX)):

            #limit YX dataframe
            YX_lim = YX[YX.index[t-m]:YX.index[t]]

            #taking Y and X series for regression
            Y_lim = YX_lim[YX_lim.columns[0]]
            X_lim = YX_lim.drop(YX_lim.columns[0], axis=1)

            #fit the model with a constant
            X1 = sm.add_constant(X_lim)
            reg = sm.OLS(Y_lim, X1).fit()

            #taking predictions
            fit = reg.predict()
            fit = pd.Series(fit, name = 'Predictions', index = Y_lim.index)

            #append rolling predictions
            rolling_predictions[t-m] = fit[-1]

            #append rolling intercepts
            intercept[t-m] = reg.params[0]

            #append rolling betas
            for y in range(0, len(beta.columns)):
                beta[beta.columns[y]][t-m] = reg.params[y+1]

            #append rolling rsquared
            r2[t-m] = reg.rsquared

        #solve for errors
        errors = rolling_predictions - Y_test
        
        #put important data into a dataframe for export
        results[ccy_id + '_' + time_frame + '_' + model_id + '_' + 'actual'] = Y_test
        results = pd.concat([results, X], axis = 1).reindex(Y_test.index)
        results[ccy_id + '_' + time_frame + '_' + model_id + '_' + 'intercept'] = intercept
        results = pd.concat([results, beta], axis = 1)
        results[ccy_id + '_' + time_frame + '_' + model_id + '_' + 'r2'] = r2
        results[ccy_id + '_' + time_frame + '_' + model_id + '_' + 'prediction'] = rolling_predictions
        results[ccy_id + '_' + time_frame + '_' + model_id + '_' + 'error'] = errors
    
    return results





****

## Full period regressions

In [ ]:
#full period regressions for G10 pairs
for r in range(len(map)):
    ccy_pair = map['g10_pairs'][r]
    Y = df[map['g10_pairs'][r] + ' curncy']
    print(ccy_pair)
    
    #model combinations
    X1 = df[['2Y spread ' +  map['g10_pairs'][r]]]
    X2 = df[['10Y BTP spread']]
    X3 = df[['US 2s10s']]
    X4 = df[['brent']]
    X5 = df[['2Y spread ' +  map['g10_pairs'][r],'10Y BTP spread' ]]
    X6 = df[['2Y spread ' +  map['g10_pairs'][r],'US 2s10s' ]]
    X7 = df[['2Y spread ' +  map['g10_pairs'][r],'brent' ]]
    X8 = df[['10Y BTP spread','US 2s10s' ]]
    X9 = df[['10Y BTP spread','brent' ]]
    X10 = df[['US 2s10s', 'brent']]
    X11 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'US 2s10s']]
    X12 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'brent']]
    X13 = df[['2Y spread ' +  map['g10_pairs'][r], 'US 2s10s', 'brent']]
    X14 = df[['10Y BTP spread', 'US 2s10s', 'brent']]
    X15 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'US 2s10s', 'brent']]

    #make a list of models
    X_list = [X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15]
    
    #regress the models
    for x in range(len(X_list)):
        regress(Y, X_list[x])

## Rolling regressions

In [ ]:
#rolling regressions for G10 pairs (first half)

#create output dataframe
output = pd.DataFrame()

#loops for G10 pairs
for r in range(round(len(map)/2)):
    ccy_pair = map['g10_pairs'][r]
    Y = df[map['g10_pairs'][r] + ' curncy']
    
    #model combinations
    X1 = df[['2Y spread ' +  map['g10_pairs'][r]]]
    X2 = df[['10Y BTP spread']]
    X3 = df[['US 2s10s']]
    X4 = df[['brent']]
    X5 = df[['2Y spread ' +  map['g10_pairs'][r],'10Y BTP spread' ]]
    X6 = df[['2Y spread ' +  map['g10_pairs'][r],'US 2s10s' ]]
    X7 = df[['2Y spread ' +  map['g10_pairs'][r],'brent' ]]
    X8 = df[['10Y BTP spread','US 2s10s' ]]
    X9 = df[['10Y BTP spread','brent' ]]
    X10 = df[['US 2s10s', 'brent']]
    X11 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'US 2s10s']]
    X12 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'brent']]
    X13 = df[['2Y spread ' +  map['g10_pairs'][r], 'US 2s10s', 'brent']]
    X14 = df[['10Y BTP spread', 'US 2s10s', 'brent']]
    X15 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'US 2s10s', 'brent']]

    #make a list of models
    X_list = [X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15]
    
    #loops for models
    for x in range(len(X_list)):
        output = pd.concat([output, rolling_reg(Y, X_list[x], ccy_pair, 'X' + str(x+1))], axis=1)

In [ ]:
#rolling regressions for G10 pairs (second half)


#loops for G10 pairs
for r in range(round(len(map)/2), len(map)):
    ccy_pair = map['g10_pairs'][r]
    Y = df[map['g10_pairs'][r] + ' curncy']
    
    #model combinations
    X1 = df[['2Y spread ' +  map['g10_pairs'][r]]]
    X2 = df[['10Y BTP spread']]
    X3 = df[['US 2s10s']]
    X4 = df[['brent']]
    X5 = df[['2Y spread ' +  map['g10_pairs'][r],'10Y BTP spread' ]]
    X6 = df[['2Y spread ' +  map['g10_pairs'][r],'US 2s10s' ]]
    X7 = df[['2Y spread ' +  map['g10_pairs'][r],'brent' ]]
    X8 = df[['10Y BTP spread','US 2s10s' ]]
    X9 = df[['10Y BTP spread','brent' ]]
    X10 = df[['US 2s10s', 'brent']]
    X11 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'US 2s10s']]
    X12 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'brent']]
    X13 = df[['2Y spread ' +  map['g10_pairs'][r], 'US 2s10s', 'brent']]
    X14 = df[['10Y BTP spread', 'US 2s10s', 'brent']]
    X15 = df[['2Y spread ' +  map['g10_pairs'][r], '10Y BTP spread', 'US 2s10s', 'brent']]

    #make a list of models
    X_list = [X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15]
    
    #loops for models
    for x in range(len(X_list)):
        output = pd.concat([output, rolling_reg(Y, X_list[x], ccy_pair, 'X' + str(x+1))], axis=1)

In [ ]:
output.to_csv('output.csv')

****

## Plot rolling model fits

In [ ]:
#inputs
ccy_pair = 'usdeur'
time_frame = '2y'   #choices are: 3m, 6m, 1y, 2y, 4y, grow

#plot models
for x in range(len(X_list)):
    plt.figure(figsize = (20,7))
    plt.plot(output[ccy_pair + '_' + time_frame + '_X1_actual'], color = 'red')
    plt.plot(output[ccy_pair + '_' + time_frame + '_X' + str(x+1) + '_prediction'], color = 'blue')
    plt.legend(('Actual', 'Model Fit'), fontsize = 10)
    plt.title('Model ' + str(x+1), fontsize = 20)
    plt.show()

## Alan Notes

In [ ]:
### cut the pairs in  2 GROUPS
## ADD THE DIFFERENT TRIME HROOIZON
## add title to each graph and labels and 
### lets add the f statiscs asn an additional column
### accumualated residuals rolling
### save the last charts ### create JPG
#SAVE AS PDF MERGED
## add timeclocks
###only process marginal deltan

In [ ]:
#### multuipole loops

n=0
for w in range(len(time_ref_list)): 
    print("time loop")
    print("w "+str(w))
    time_ref_start_name=str(time_ref_start[w]) 
    time_ref_end_name=str(time_ref_end[w]) 
    time_ref_name=str(time_ref_start_name)+" / "+str(time_ref_end_name)

    #####filter time set here
    for x in range(len(x0_list)): 
        print("x loop")
        print("x "+str(x))
        x_variable=str(x0_list[x])
        print(x_variable)
        ###fitlere variable here
        for y in range(len(y0_list)): 
            ###fitlere variable here
            print("y loop")
            print("y "+str(y))
            y_variable=str(y0_list[y])
            print(y_variable)
         
 

In [ ]:
#Add tittle
### speciifc the variables of the medeil in the title or subtittle.


fig.suptitle("EM Global Macro Sensitivity Analysis: "+str(x_variable)+" vs. "+str(y_variable)+" Time: ("+str(time_ref_name)+")",size = 15)
            plt.title('Interpretation: t-stat ->  significance // reg coefficient -> sensitivity ', size=parameter_l)
            plt.ylabel(y_label, size=parameter_l)
            plt.xlabel(x_label, size=parameter_l)       

In [ ]:
## save as jpgs



        plt.savefig(str(y_var)+"/table_"+str(y_var)+"th_"+str(th)+".jpg", bbox_inches='tight')
        plt.show()
        plt.close()
        plt.close('all')
        plt.clf()
        plt.cla()
        
  

In [ ]:
###merge all pdfs to one fielm 

source_dir= 'K://table/'
source_dir2= 'K://Summary/'
 

g0_list=['dv_cds5y_','dv_fx_','dv_l_msci_','dv_lccy_','dv_hccy_']  
#g0_list=['dv_cds5y_','dv_fx_','dv_l_msci_']
#y0_list=['dv_cds5y_','dv_fx_','dv_l_msci_','dv_lccy_','dv_hccy_']

#####filter time set here
for x in range(len(g0_list)): 
        merger = PdfFileMerger()
        variable=str(g0_list[x])
        variable_folder = str(variable)+'/'
        source_dir1 = str(source_dir) + str(variable_folder) +"/"
        print(variable)
        print(source_dir1)
        for item in os.listdir(source_dir1):
             if item.endswith('pdf'):
                merger.append(source_dir1 + item)

        merger.write(source_dir2+variable+"_Complete.pdf") 
        #merger.write(public_location+variable+"_Complete.pdf")       
        merger.close()
        
        
        
        
print(table_pv)
print("Cmplete")
now = datetime.now()
print("Y",now.year,"/ M", now.month,"/ D", now.day, "/// H",now.hour,": M", now.minute,": S", now.second)
#timer: 101.14274072647095 seconds++

In [ ]:
### acumualrte the residual

list_loop=list_x1
for t in range(len(time)):
    t1=str(time[t])
    print(t1)
    for x in range(len(list_loop)):
        print(t1)
        print(x)
        y=str(list_loop[x])      
        f=str("factor_reg_"+str(y)) 
        print(f)
        df[str('acumm_factor_'+str(t1))]= df[str(f)].rolling(window=rolling_window2).agg(lambda x : x.prod())
        df.rename(columns={str('acumm_factor_'+str(t1)):(str('acumm_factor_'+str(t1))+"_"+str(y))}, inplace=True)
        ff = f[-6:]
        ff1=("lg"+str(t1)+"_"+str(ff))
        print(ff)
        print(ff1)   
        df["output"]=(df["acumm_factor_"+str(t1)+"_"+str(y)])+0
        df["output0"]=(df[str(ff1)])
        df["output"]=df["output"]*df["output0"]
        del df["output0"]
        df.rename(columns={"output":("output_"+str(t1)+"_"+str(y))}, inplace=True)   
        df["acumm_res"]=((df[("output_"+str(t1)+"_"+str(y))]/df[str(ff)])-1)
        df.rename(columns={'acumm_res':("acumm_res_"+str(t1)+"_"+str(y))}, inplace=True)    